# Problem description

This is an analysis for the DrivenData competition on predicting Heart Disease: https://www.drivendata.org/competitions/54/machine-learning-with-a-heart/page/107/

The goal is to predict the binary class ```heart_disease_present```, which represents whether or not a patient has heart disease:

- ```0``` represents no heart disease present
- ```1``` represents heart disease present

There are 14 columns in the dataset, where the ```patient_id``` column is a unique and random identifier. The remaining 13 features are described in the section below.
- ```slope_of_peak_exercise_st_segment``` (type: int): the slope of the peak exercise ST segment, an electrocardiography read out indicating quality of blood flow to the heart
- ```thal``` (type: categorical): results of thallium stress test measuring blood flow to the heart, with possible values ```normal```, ```fixed_defect```, ```reversible_defect```
- ```resting_blood_pressure``` (type: int): resting blood pressure
- ```chest_pain_type``` (type: int): chest pain type (4 values)
- ```num_major_vessels``` (type: int): number of major vessels (0-3) colored by flourosopy
- ```fasting_blood_sugar_gt_120_mg_per_dl``` (type: binary): fasting blood sugar > 120 mg/dl
- ```resting_ekg_results``` (type: int): resting electrocardiographic results (values 0,1,2)
- ```serum_cholesterol_mg_per_dl``` (type: int): serum cholestoral in mg/dl
- ```oldpeak_eq_st_depression``` (type: float): oldpeak = ST depression induced by exercise relative to - rest, a measure of abnormality in electrocardiograms
- ```sex``` (type: binary): ```0```: female, ```1```: male
- ```age``` (type: int): age in years
- ```max_heart_rate_achieved``` (type: int): maximum heart rate achieved (beats per minute)
- ```exercise_induced_angina``` (type: binary): exercise-induced chest pain (```0```: False, ```1```: True)

Performance is evaluated according to binary log loss.

The format for the submission file is two columns with the ```patient_id``` and ```heart_disease_present```. This competition uses log loss as its evaluation metric, so the ```heart_disease_present``` values you should submit are the probabilities that a patient has heart disease (not the binary label).

# Preparation of Environment

## Get the required libraries

In [ ]:
# Load required packages

import urllib.request
import os

import pandas as pd
import numpy as np
from scipy import stats
import math

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

#%matplotlib inline

## Get the data

In [ ]:
#import define and create the data folder
data_folder = os.path.join(os.getcwd(), 'Data')
os.makedirs(data_folder, exist_ok=True)

In [ ]:
#download the data
urllib.request.urlretrieve('https://s3.amazonaws.com/drivendata/data/54/public/train_values.csv', filename=os.path.join(data_folder, 'train-features.csv'))
urllib.request.urlretrieve('https://s3.amazonaws.com/drivendata/data/54/public/train_labels.csv', filename=os.path.join(data_folder, 'train-labels.csv'))
urllib.request.urlretrieve('https://s3.amazonaws.com/drivendata/data/54/public/test_values.csv', filename=os.path.join(data_folder, 'test-features.csv'))
urllib.request.urlretrieve('https://s3.amazonaws.com/drivendata/data/54/public/submission_format.csv', filename=os.path.join(data_folder, 'submission-format.csv'))

In [ ]:
#import the data into the notebook and defined first column as index (patient id)
train_features = pd.read_csv(os.path.join(data_folder, 'train-features.csv'),index_col=0)
train_labels = pd.read_csv(os.path.join(data_folder, 'train-labels.csv'), index_col=0)
test_features = pd.read_csv(os.path.join(data_folder, 'test-features.csv'), index_col=0)
submission_format = pd.read_csv(os.path.join(data_folder, 'submission-format.csv'), index_col=0)

In [ ]:
#create one training dataframe

train_features['heart_disease_present'] = train_labels['heart_disease_present']

heart = train_features

# Data Preparation

In [ ]:
#check how the first rows of the dataset look like
heart.head()

In [ ]:
#check the types of the columns
heart.dtypes

In [ ]:
#create lists of categorical, numerical and label columns
catfeat = ['slope_of_peak_exercise_st_segment', 'thal', 'chest_pain_type', 'num_major_vessels', 'fasting_blood_sugar_gt_120_mg_per_dl', 'resting_ekg_results', 'sex']
numfeat = [ 'resting_blood_pressure', 'serum_cholesterol_mg_per_dl', 'oldpeak_eq_st_depression', 'age', 'max_heart_rate_achieved']
label = 'heart_disease_present'

In [ ]:
#convert categorical columns to categories
for col in catfeat:
    heart[col] = heart[col].astype('category')

In [ ]:
#check if it worked
heart.dtypes

In [ ]:
#check the content of the colums again
heart.head()

In [ ]:
#check for missing values
print("Missings coded as NAs: \n", heart.isnull().any())

print("\n Missings coded as ?: \n", (heart.astype(np.object) == '?').any())

In [ ]:
#check for duplicate rows
print(heart.shape)
print(heart.index.unique().shape)

## Vizualization

### Numerical Features

In [ ]:
#investigate descriptive statistics for numeric features
heart[numfeat].describe()

In [ ]:
#investigate distribution propoerties kurtosis and skewness of numeric features

for col in numfeat: 
    print(col, ': \nexcess kurtosis (should be 0): {}'.format(stats.kurtosis(heart[col])))
    print('skewness of (should be 0): {}'.format(stats.skew(heart[col])))
    print('\n')

In [ ]:
#Visualize numeric feature distribution with displots 
def hist(df, cols, nbins):
    for col in cols:
        sns.distplot(df[col], bins = nbins )
        plt.xlabel(col)
        plt.show()

hist(heart, numfeat, 10)

In [ ]:
#test for normality

for col in numfeat:
    shst, p = stats.shapiro(heart[col])
    print(col, ':')
    print('Test statistic: ', shst)
    print('P-Value for: ', p, '\n')

In [ ]:
#visualize correlations between numerical features in a heat map
corr = abs(heart[numfeat].corr())
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
#Vizualize label separation by numeric features with a box plot
def plot_box(df, cols, col_x = label):
    for col in cols:
        sns.set_style("whitegrid")
        sns.boxplot(col_x, col, data=df)
        plt.xlabel(col_x) # Set text for the x axis
        plt.ylabel(col)# Set text for y axis
        plt.show()

plot_box(heart, numfeat)

In [ ]:
#test the relationships between the numerical features and the label

for col in numfeat:
    print(col, "and the label: \n", stats.pointbiserialr(heart[col], heart[label]))

### Categorical features

In [ ]:
#investigate descriptive statistics for categorical features
heart[catfeat].describe()

In [ ]:
#visualize the number of cases per category with bar charts
for col in catfeat: 
    nrcat = heart[col].value_counts()
    nrcat.plot.bar(rot=1)
    plt.ylabel(col)
    plt.show()

In [ ]:
#Vizualize label separation by categorical features with a bar charts
heart['dummy'] = np.ones(shape = heart.shape[0])

for col in catfeat:
    print(col)
    counts = heart[['dummy', label, col]].groupby([label, col], as_index = False).count()
    temp = counts[counts[label] == 0][[col, 'dummy']]
    _ = plt.figure(figsize = (10,4))
    plt.subplot(1, 2, 1)
    temp = counts[counts[label] == 0][[col, 'dummy']]
    plt.bar(temp[col], temp.dummy)
    plt.xticks(rotation=90)
    plt.title('Counts for ' + col + ' and no heart disease')
    plt.ylabel('count')
    plt.subplot(1, 2, 2)
    temp = counts[counts[label] == 1][[col, 'dummy']]
    plt.bar(temp[col], temp.dummy)
    plt.xticks(rotation=90)
    plt.title('Counts for ' + col + ' and heart disease')
    plt.ylabel('count')
    plt.show()

del heart['dummy']

In [ ]:
#test the relationships between the categorical features and the label

for col in catfeat:
    ct = pd.crosstab(heart[col], heart[label])
    print(col, ": \n", stats.chisquare(ct))

In [ ]:
#visualize class distribution of the label
heart_counts = heart[label].value_counts()
heart_sum = sum(heart_counts)
heart_perc = heart_counts / heart_sum

heart_perc.plot.bar(rot=1)

### Observations

- No missing values


- ```age``` is the only feature that is statisticall normally distributed
- The distribution of ```serum_cholesterol_mg_per_dl``` and  ```oldpeak_eq_st_depression``` differ the most from normal distributions
- ```serum_cholesterol_mg_per_dl``` contains extreme outliers, but apart from the outliers the form resembles a normal distribution
- ```oldpeak_eq_st_depression``` also contains outliers, but the form does not resemble a normal distribution even when ignoring the outliers
- numerical features have low to medium intercorrelations
- ```oldpeak_eq_st_depression``` and ```max_heart_rate_achieved```  can visually distinguish the best between heart and no heart disease, this is confirmed when calculating a point biserial correlation.
- Interestingly, ```serum_cholesterol_mg_per_dl``` does not distinguish so well between heart and no heart disease. 


- most of the categorical features have categories with very little data
- categories that distinguish well between heart and no heart disease are ```slope_of_peak_exercise_st_segment```, ```thal```, ```chest_pain_type```, ```num_major_vessels``` and ```sex```


- the number of cases in the label variables are more or less equal and thus no undersampling is required


- Thus, the features and are further investigated and taken for predictions in the first run are ```oldpeak_eq_st_depression```, ```max_heart_rate_achieved```, ```slope_of_peak_exercise_st_segment```, ```thal```, ```chest_pain_type```, ```num_major_vessels``` and ```sex```. 
- For ```resting_blood_pressure```, ```fasting_blood_sugar_gt_120_mg_per_dl```, ```resting_ekg_results```, ```serum_cholesterol_mg_per_dl```, ```age``` and ```exercise_induced_angina```, further feature engineering will be performed in a second run through.

## Transformation and Feature Engineering

### Aggregating categories

Of the requencies that distinguish well between heart and no heart disease, ```slope_of_peak_exercise_st_segment```, ```thal```, ```chest_pain_type```, ```num_major_vessels``` contain categories with a small amount of cases.

However, one must be careful with aggregating categories. This only makes sense for categories that are similar in the domain of the problem. Thus, domain expertise must be applied. 

We will leave them for now as they are.

### Transforming numeric variables

```oldpeak_eq_st_depression``` is not normally distributed.

In [ ]:
#transform 'oldpeak_eq_st_depression' with a square root and compare the distributions

heart['sqr_depression'] = np.sqrt((heart['oldpeak_eq_st_depression']))

hist(heart, ['sqr_depression', 'oldpeak_eq_st_depression'], 10)

### Compute new features

In [ ]:
#create a new categorical feature called 'rng_depression' from 'oldpeak_eq_st_depression'

maxs = heart['oldpeak_eq_st_depression'].max()
bin = [-1, 1 ,maxs]

heart['rng_depression'] = pd.cut(heart['oldpeak_eq_st_depression'],bin)

heart.head()

In [ ]:
#visualize counts of rng_depression categories with a bar chart

rng_dep_counts = heart['rng_depression'].value_counts()

rng_dep_counts.plot.bar(rot=1)

### Observations

- log-transforming ```oldpeak_eq_st_depression``` is not suitable as it contains predominantly 0 values
- transforming ```oldpeak_eq_st_depression``` by applying the square root resulted in a distribution that seemed to contain two subdistributions
- ```oldpeak_eq_st_depression``` is thus converted into the categorical feature ```rng_depression``` with two categories: cases with a 0-level of depression and cases with a level higher than 0 of depression


- Features taken for building a model are ```rng_depression```, ```max_heart_rate_achieved```, ```slope_of_peak_exercise_st_segment```, ```thal```, ```chest_pain_type```, ```num_major_vessels``` and ```sex```.

# Local modelling

## Preparing data for scikit

In [ ]:
#define categorical and numerical features used for modelling
sel_catfeat = ['rng_depression', 'slope_of_peak_exercise_st_segment', 'thal', 'chest_pain_type', 'num_major_vessels', 'sex']
sel_numfeat = ['max_heart_rate_achieved']

In [ ]:
#One hot encode categorical features and concatenate them all into a new df

heart_prep = pd.DataFrame(heart.index).set_index('patient_id')

for col in sel_catfeat:
    temp = pd.DataFrame(pd.get_dummies(heart[col], prefix=col))
    heart_prep = pd.concat([heart_prep, temp], axis=1, join_axes=[heart_prep.index])

heart_prep.head()

In [ ]:
#add selected numerical features to new df
heart_prep[sel_numfeat] = heart[sel_numfeat]

In [ ]:
#standardize numerical features
for col in sel_numfeat: 
    heart_prep[col] = preprocessing.scale(heart_prep[col])

In [ ]:
#save the prepared dataset in a file


In [ ]:
#create a numpy array of train features
np_train_feat = np.array(heart_prep)

In [ ]:
#create a numpy array of the label
np_train_label = np.array(heart[label])

In [ ]:
#split the new dataset into a training and test dataset and check their shapes
x_train, x_test, y_train, y_test = ms.train_test_split(np_train_feat, np_train_label, test_size=0.33, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

## Create and test a model using logistic regression

In [ ]:
#train the model
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(x_train, y_train)

In [ ]:
#create predictions with the model
probabilities = logistic_mod.predict_proba(x_test)

In [ ]:
#transform probabilities into class scores
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

scores = score_model(probabilities, 0.5)
print(np.array(scores[:15]))
print(y_test[:15])

In [ ]:
#evaluate the predictions with a confusion matrix, accuracy, precision, recall and F1
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])

   
print_metrics(y_test, scores)    

In [ ]:
#calculate the log loss evaluation metric
sklm.log_loss(y_test, scores)

In [ ]:
#evaluate the predictions with the ROC curve and AUC
def plot_auc(labels, probs):
    ## Compute the false positive rate, true positive rate
    ## and threshold along with the AUC
    fpr, tpr, threshold = sklm.roc_curve(labels, probs[:,1])
    auc = sklm.auc(fpr, tpr)
    
    ## Plot the result
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
plot_auc(y_test, probabilities)    

## Model selection

# Modelling in AMLS

# Modelling with AutoML

# Submission

In [ ]:
test_features.head()

In [ ]:
#define a function to do all the preprocessing with the test data
def preprocess(df):
    #define categorical and numerical features used for modelling
    sel_catfeat = ['rng_depression', 'slope_of_peak_exercise_st_segment', 'thal', 'chest_pain_type', 'num_major_vessels', 'sex']
    sel_numfeat = ['max_heart_rate_achieved']
    
    #create a new categorical feature called 'rng_depression' from 'oldpeak_eq_st_depression'
    maxs = df['oldpeak_eq_st_depression'].max()
    bin = [-1, 1 ,maxs]
    df['rng_depression'] = pd.cut(df['oldpeak_eq_st_depression'],bin)
    
    #One hot encode categorical features and concatenate them all into a new df
    df_prep = pd.DataFrame(df.index).set_index('patient_id')
    for col in sel_catfeat:
        temp = pd.DataFrame(pd.get_dummies(df[col], prefix=col))
        df_prep = pd.concat([df_prep, temp], axis=1, join_axes=[df_prep.index])
        
    #add selected numerical features to new df
    df_prep[sel_numfeat] = df[sel_numfeat]
    
    #standardize numerical features
    for col in sel_numfeat: 
        df_prep[col] = preprocessing.scale(df_prep[col])
    
    #create a numpy array of train features
    np_prep = np.array(df_prep)
    
    return np_prep
    
test_features_proc = preprocess(test_features)

In [ ]:
#check if the preprocessing worked
print(test_features_proc)

In [ ]:
#create predictions with the model (Do not convert to binary labels, submissions must be made with probabilities)
submission_probs = logistic_mod.predict_proba(test_features_proc)

In [ ]:
#add the predictions to the submission file and save it as csv
submission_format['heart_disease_present'] = submission_probs

submission_format.to_csv(os.path.join(data_folder, 'test-predictions.csv'))